<a href="https://colab.research.google.com/github/jnrkufuor/apollo/blob/Eric/Copy_of_NER_RH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Packages

In [40]:
# install flair
!pip install flair

# load basic packages
import pandas as pd
from itertools import combinations, product
import string
import re
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# load Flair and NLTK
import torch
from flair.data import Sentence
from flair.models import SequenceTagger
from nltk import tokenize

import nltk
nltk.download('punkt')

# is cuda available?
torch.cuda.is_available()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


False

## 2. Load Flair NER model

In [3]:
#Load NER Model
tagger = SequenceTagger.load('ner')

2021-03-19 06:29:49,095 --------------------------------------------------------------------------------
2021-03-19 06:29:49,097 The model key 'ner' now maps to 'https://huggingface.co/flair/ner-english' on the HuggingFace ModelHub
2021-03-19 06:29:49,098  - The most current version of the model is automatically downloaded from there.
2021-03-19 06:29:49,099  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt)
2021-03-19 06:29:49,100 --------------------------------------------------------------------------------



2021-03-19 06:30:00,164 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4


## 3. Load Data

In [7]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/googlenews1000.csv')
#Above two lines may have to be changed to import your data depending on where it is
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,title,media,date,datetime,desc,link,img,content
0,0,STUCK IN WALMART PARKING lot after Colorado storm,FOX31 Denver,1 hour ago,26:32.8,Several drivers were forced into a Walmart par...,https://kdvr.com/news/local/drivers-spend-hour...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","AURORA, Colo. (KDVR) — The historic snowstorm ..."
1,1,Suspect wanted in fatal Monkey Junction Walmar...,Home - WSFX,1 hour ago,26:32.8,Laron Lee Carter is wanted by the NHC Sheriff'...,https://foxwilmington.com/local-news/suspect-w...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",According to a spokesperson for the New Hanove...
2,2,Walmart Theft Investigation Turns Into Chase A...,KVRR,1 hour ago,26:32.8,— Two people are arrested after a chase where ...,https://www.kvrr.com/2021/03/15/walmart-theft-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Walmart Theft Investigation Turns Into Chase A...
3,3,Walmart Calls Ex-Worker's 'Stonewalling' Claim...,NaN,1 hour ago,26:32.8,"Law360 (March 15, 2021, 5:49 PM EDT) -- An ex-...",https://www.law360.com/retail/articles/1364916...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Law360 (March 15, 2021, 5:49 PM EDT) -- An ex-..."
4,4,Three wanted for questioning in Walmart shopli...,wgxa.tv,1 hour ago,26:32.8,"HOUSTON COUNTY, Ga. -- Perry police are lookin...",https://wgxa.tv/news/local/three-wanted-for-qu...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",NaN
...,...,...,...,...,...,...,...,...,...
94,5,Biopharmaceutical Logistic Market 2021 by Comp...,NaN,10 hours ago,26:45.2,"... Type and Application, Forecast to 2025 : B...",https://jumbonews.co.uk/uncategorised/2211131/...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The research report by OrbisPharmaReports on g...
95,6,Pharma making steady progress in meeting 2023 ...,European Pharmaceutical Review,10 hours ago,26:45.2,The study was conducted in 2020 by Amerisource...,https://www.europeanpharmaceuticalreview.com/n...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The GS1 US Barcode Assessment study indicates ...
96,7,Pharmacy Automation & Medical Supplies Automat...,NaN,12 hours ago,26:45.2,"... Recent Trends, Growth Factors, Business Op...",https://www.mccourier.com/pharmacy-automation-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Global Pharmacy Automation & Medical Supplies ...
97,8,Vaccine Storage & Packaging Market Growth Anal...,KSU | The Sentinel Newspaper,12 hours ago,26:45.2,"Lineage Logistics, DB Schenker, AmerisourceBer...",https://ksusentinel.com/2021/03/15/vaccine-sto...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",(March 2021) The Global Vaccine Storage & Pack...


#4. Remove pronouns

In [8]:

pronouns = ['I', 'You', 'It', 'He', 'She', 'We', 'They']
suffixes = ["", "’m", "’re", "’s", "’ve", "’d", "'m", "'re", "'s", "'ve", "'d", "m", "re", "s", "ve", "d"]

contraptions = [(p, s) for p in pronouns for s in suffixes]

df_contraptions = pd.DataFrame(contraptions, columns=['pronoun', 'suffix'])

df_contraptions['contraption'] = df_contraptions.apply(lambda x: x['pronoun'] + x['suffix'], axis=1)

contraptions = df_contraptions.contraption.values


## 4. Define NER function

In [9]:
# define function

def get_ner_data(paragraph):
    '''
    - function to extract named entities from a paragraph
    - returns two data frames:
        - the first is a dataframe of all unique entities (persons and orgs)
        - the second is the links between the entities
    '''
    
    # remove newlines and odd characters
    paragraph = re.sub('\r', '', paragraph)
    paragraph = re.sub('\n', ' ', paragraph)
    paragraph = re.sub("’s", '', paragraph)
    paragraph = re.sub("“", '', paragraph)
    paragraph = re.sub("”", '', paragraph)

    
    # tokenise sentences
    sentences = tokenize.sent_tokenize(paragraph)
    sentences = [Sentence(sent) for sent in sentences]
    
    # predict named entities
    for sent in sentences:
        tagger.predict(sent)
    
    # collect sentence NER's to list of dictionaries
    sent_dicts = [sentence.to_dict(tag_type='ner') for sentence in sentences]
    
    # collect entities and types
    entities = []
    types = []
    for sent_dict in sent_dicts:
        entities.extend([entity['text'] for entity in sent_dict['entities']])
        types.extend([str(entity['labels'])[1:4] for entity in sent_dict['entities']])
   #The above line is what I changed from the default notebook to get things working     
    
    # create dataframe of entities (nodes)
    df_ner = pd.DataFrame(data={'entity': entities, 'type': types})
    df_ner = df_ner[df_ner['type'].isin(['ORG'])]
    df_ner = df_ner[df_ner['entity'].map(lambda x: isinstance(x, str))]
    df_ner = df_ner[~df_ner['entity'].isin(df_contraptions['contraption'].values)]
    df_ner['entity'] = df_ner['entity'].map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    #df_ner['entity'] = df_ner.apply(lambda x: x['entity'].split(' ')[len(x['entity'].split(' '))-1] if x['type']=='PER' else x['entity'], axis=1)
    df_ner = df_ner.drop_duplicates().sort_values('entity')
    
    # get entity combinations
    combs = list(combinations(df_ner['entity'], 2))
    
    # create dataframe of relationships (edges)
    df_links = pd.DataFrame(data=combs, columns=['from', 'to'])
    
    return df_ner, df_links

## 5. Apply function

In [22]:
#df['date'] = pd.to_datetime(df['date'])
#day = '2021-01-24'
#df_day = df[df['date']==pd.to_datetime(day)][['date', 'domain', 'title', 'content']]


In [23]:
df_domain = df.groupby('media').agg({'content': 'count'}).reset_index()
df_domain.columns = ['media', 'count']
df_domain = df_domain.sort_values('count', ascending=False)
dfd_small=df_domain.iloc[1:21,:]

dfd_small

# g2 = sns.barplot(data=dfd_small,
#              x='count',
#              y='Domain',
#              dodge=False,
#              orient='h',
#              hue='count',
#              palette='viridis')

# g2.set_yticks([])
# g2.set_title('Number of articles from each provider')
# g2.set_xlabel('Count')
# g2.set_ylabel('')
# g2.set_xlim(0, max(dfd_small['count'])+150)
# g2.legend_.remove()
# g2.tick_params(labelsize=5)

# for i in dfd_small.index:
#             g2.text(df_domain.iloc[i]['count']+5, i+0.25, df_domain.iloc[i]['Domain'], fontsize=8)

# sns.despine()
# g2.get_figure().savefig('domain_plot.png', dpi=1000)

,media,count
51,Weatherford Democrat,2
36,MarketWatch,2
41,SoccerNurds,2
0,9to5Mac,1
43,Stocks Register,1
31,JD Supra,1
32,Jumbo News,1
34,KVRR,1
35,Mac Rumors,1
37,Paris Post Intelligencer,1


In [11]:
df_ner = pd.DataFrame()
df_links = pd.DataFrame()

for content in tqdm(df['content']):
  try:
    df_ner_temp, df_links_temp = get_ner_data(content)

    df_ner = df_ner.append(df_ner_temp)
    df_links = df_links.append(df_links_temp)
  except:
    continue



100%|██████████| 99/99 [1:20:16<00:00, 48.65s/it]


In [29]:
praph=df['content'].iloc[1]
praph = re.sub('\r', '', praph)
praph = re.sub('\n', ' ', praph)
praph = re.sub("’s", '', praph)
praph = re.sub("“", '', praph)
praph = re.sub("”", '', praph)

    
# tokenise sentences
sentences = tokenize.sent_tokenize(praph)
sentences = [Sentence(sent) for sent in sentences]

## predict named entities
for sent in sentences:
    tagger.predict(sent)
    
# # collect sentence NER's to list of dictionaries
sent_dicts = [sentence.to_dict(tag_type='ner') for sentence in sentences]



# # collect entities and types
entities = []
types = []
for sent_dict in sent_dicts:
    entities.extend([entity['text'] for entity in sent_dict['entities']])
    types.extend([str(entity['labels'])[1:4] for entity in sent_dict['entities']])

#types

# 6. Remove plurals and possessives

In [16]:
def remove_s(entity, entity_series):
  if (entity[-1] == 's') & (entity[:-1] in entity_series):
    return entity[:-1]
  else:
    return entity



In [13]:
df_links['to'] = df_links['to'].map(lambda x: remove_s(x, df_ner['entity'].values))
df_links['from'] = df_links['from'].map(lambda x: remove_s(x, df_ner['entity'].values))
df_ner['entity_cl'] = df_ner['entity'].map(lambda x: remove_s(x, df_ner['entity'].values))



In [26]:
df_links[df_links['to'].str.contains('They')]

,from,to


# 7. Export Data

In [17]:
df_ner.to_csv('/content/drive/My Drive/df_ner_att1.csv', index=False)
df_links.to_csv('/content/drive/My Drive/df_links_att1.csv', index=False)

#Use the above two lines to write the critical dataframes to csv files in your 
#google drive account

In [42]:
from google.colab import drive
drive.mount('/content/drive')
df_links_att1 = pd.read_csv('/content/drive/My Drive/df_links_att1.csv')
df_ner_att1 = pd.read_csv('/content/drive/My Drive/df_ner_att1.csv')
#Above two lines may have to be changed to import your data depending on where it is
#df_links_att1
#df_ner_att1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
